<a href="https://colab.research.google.com/github/harrisontg57/NLP_Spring_2021/blob/main/Exam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tim Harrison
Exam 1
Student ID:

Question 1)

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

def create_model(alg_name, vectorization, training_data):
  if alg_name == "naive_bayes":
    learn = MultinomialNB()
  elif alg_name == "perceptron":
    learn = Perceptron()
  elif alg_name == "logistic_regression":
    learn = LogisticRegression()
  else:
    raise NameError('incorrect algorithm name')

  if vectorization == "tfidf":
    vect = TfidfVectorizer()
  elif vectorization == "bag_of_words":
    vect = CountVectorizer()
  else:
    raise NameError('incorrect vectorization name')
  
  model = make_pipeline(vect, learn)
  return
